# Sonata Lightning Tools: Matadata Parser

### Program to parse the SAPBO Reports metadata for analysis

In [1]:
import requests
from lxml import etree
import json
import csv
import pandas as pd
import configparser
%run lightning_helper.ipynb

## Config class to get the BO Server configurations

In [2]:
class BOServerConfig:
    # Create a configuration parser
    config = configparser.ConfigParser()
    config.read(get_full_file_path('config','config.ini'))
    
    # Expose configuration values as class attributes
    protocol = config.get('sapboserver', 'protocol')
    ipaddress = config.get('sapboserver', 'ipaddress')
    port = config.get('sapboserver', 'port')
    username = config.get('sapboserver', 'username')
    password = config.get('sapboserver', 'password')

## Parser class to connect to SAP BO and provide login and request/response services

In [15]:
class SAP_BO_Parser():

    def __init__(self, protocol='http', host='localhost', port='8080', content_type='application/json'):
        base_url = protocol + '://' + host + ':' + port
        self.bip_url = base_url + '/biprws'
        self.webi_url = self.bip_url + '/raylight/v1'
        self.sl_url = self.bip_url + '/sl/v1'
        self.headers = {
            'Accept': 'application/json',
            'Content-Type': 'application/json'
        }
      
    def _get_auth_info(self):
        return requests.get(self.bip_url + '/logon/long',
                            headers=self.headers)

    def _send_auth_info(self, username, password):
        '''Helper function to retrieve a log in token'''
        auth_template = self._get_auth_info().text
         
        jdata = json.loads(auth_template)
        jdata["userName"] = username
        jdata["password"] = password

        return requests.post(self.bip_url + '/logon/long',headers=self.headers, data=json.dumps(jdata))     
    
    def set_logon_token(self, username, password):
        resp = self._send_auth_info(username, password)
       
        if resp.status_code == 200:
            login_response = json.loads(resp.text)
            
            # Set logon token in headers
            self.headers['X-SAP-LogonToken'] = login_response['logonToken']
        else:
            
            # Crude exception handling
            raise Exception("Could not log on and set the logon token!")
            
    def get_metadata(self, meta_url):
        rest_url = self.webi_url + '/' + meta_url
        print(rest_url)
        resp = requests.get(rest_url, headers=self.headers)
            
        if resp.status_code == 200:
            #print(resp.text)
            response = json.loads(resp.text)
            return response
        else:
            # Crude Exception handling
            raise Exception('Request Exception' + resp.status_code + ' ' + resp.reason)
    


## Writer class to write data from SAP BO into csv files for processing

In [4]:
class SAP_BO_Metadata_Writer:
    
    def convert_json_to_csv(json_rows, csv_file_path):
        
        print( json_rows[0].keys())
        column_names = json_rows[0].keys()
        print( json_rows[0])

        # Write JSON data to a CSV file
        with open(csv_file_path, mode='w', newline='') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=column_names)
            # Write header row
            writer.writeheader()  

            # Write data rows
            writer.writerows(json_rows)  
    def convert_json_to_csv_panda(json_rows, csv_file_path):
       
        df = pd.DataFrame(json_rows)
        # Write data to a CSV file
        df.to_csv(csv_file_path, index=False)
        
        return df
            


In [5]:
class Universes:
    Universes_Root = "universes"
    Universe_Root = "universe"
        
    def write_universes(json_data):
        csv_file_path = get_full_file_path('output', Universes.Universes_Root + '.csv')
        json_rows = json_data[Universes.Universes_Root][Universes.Universe_Root]
        df = SAP_BO_Metadata_Writer.convert_json_to_csv_panda(json_rows, csv_file_path);
        return df
    
    def write_universe(json_data, csv_file_path):
        csv_file_path = get_full_file_path('output', Universes.Universe_Root + '.csv')
        json_rows = json_data[Universes.Universe_Root]['universe']
        SAP_BO_Metadata_Writer.convert_json_to_csv_panda(json_rows, csv_file_path);

### Connect to BO to get the Logon Token

In [16]:
parser = SAP_BO_Parser(BOServerConfig.protocol, BOServerConfig.ipaddress, BOServerConfig.port, "application/json")
parser.set_logon_token(BOServerConfig.username, BOServerConfig.password)

### Write Universes

In [8]:
response_json = parser.get_metadata(Universes.Universes_Root)

In [9]:
df = Universes.write_universes(response_json)

In [10]:
df.head()

,id,cuid,name,type,subType,folderId,revision,description
0,6816,ARVjUijBiI1ClmVC2H.XYgc,BI platform CMS system database.unx,unx,UnxRelational,6819,423003,NaN
1,5636,AVy_0kTnql9IpZ167YoP5SA,eFashion.unx,unx,UnxRelational,5639,9,eFashion retail Data Warehouse dated 14 Oct 20...
2,6845,AfgXNuog4EVGtRaVOyhw0rA,Promotion Universe,unv,UnvClassic,6158,1,NaN
3,5646,AamfjLdF3Y1IoWCqLq7LobA,Rio2016.unx,unx,UnxRelational,5639,4,NaN
4,6159,Abk2MhnEU7NDsOrWiXmVQFs,Univers2,unv,UnvClassic,6158,1,NaN
